# Fetch and Store Historical Bitcoin Data in PostgreSQL

This notebook demonstrates how to fetch historical Bitcoin (BTC) data from the CryptoCompare API, process the data, and store it in a PostgreSQL database. The process is broken down into the following steps:

## Steps:
1. **Load Environment Variables**: We load API keys and database URL from an `.env` file.
2. **Configure Database Connection**: We set up a connection to the PostgreSQL database using SQLAlchemy.
3. **API Interaction**: We fetch historical data for Bitcoin in chunks, using the CryptoCompare API.
4. **Data Processing**: The data is processed, including renaming columns, converting timestamps, and handling duplicates.
5. **Store in Database**: The processed data is stored in a PostgreSQL database table.

### Required Libraries:
- `os`: For handling environment variables.
- `time`: For managing time intervals.
- `pandas`: For data manipulation and storage.
- `requests`: For making HTTP requests to the CryptoCompare API.
- `dotenv`: For loading environment variables from the `.env` file.
- `sqlalchemy`: For connecting to the PostgreSQL database.


In [2]:
! pip install python-dotenv requests pandas sqlalchemy psycopg2-binary


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 6.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 6.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 606.0/606.0 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 16.8 MB/s eta 0:00:0000:0100:01


In [2]:
import os
import time

import pandas as pd
import requests
from dotenv import load_dotenv
from sqlalchemy import create_engine

In [3]:
# Load Env variables
load_dotenv()  
API_KEY      = os.getenv("API_KEY")
DATABASE_URL = os.getenv("DATABASE_URL")


if not API_KEY:
    raise EnvironmentError("API_KEY missing.")
if not DATABASE_URL:
    raise EnvironmentError("DATABASE_URL missing.")


### PATCH DATABASE URL FOR SQLALCHEMY ≥1.4

In [4]:
# SQLAlchemy 1.4+ rejects "postgres://"; it needs "postgresql://"
if DATABASE_URL.startswith("postgres://"):
    DATABASE_URL = DATABASE_URL.replace("postgres://", "postgresql://", 1)

# Ensure psycopg2 is installed: pip install psycopg2-binary
engine = create_engine(DATABASE_URL, echo=False, future=True)

# CONFIG
TABLE_NAME   = "bitcion_daily_data"
HISTORICAL_CHUNK = 2000  # max per-request limit
BASE_URL         = "https://min-api.cryptocompare.com/data/v2/histoday"
HEADERS          = {"authorization": f"Apikey {API_KEY}"}



## Fetching Data

In [5]:
# FETCH A CHUNK
def fetch_chunk(days: int, to_ts: int = None) -> list:
    limit_param = max(days - 1, 1)
    params = {"fsym": "BTC", "tsym": "USD", "limit": limit_param}
    if to_ts:
        params["toTs"] = to_ts

    resp = requests.get(BASE_URL, params=params, headers=HEADERS)
    resp.raise_for_status()
    data = resp.json()
    if data.get("Response") != "Success":
        raise RuntimeError(f"API Error: {data.get('Message', 'Unknown')}")
    return data["Data"]["Data"]



## Fetch Full Historical Data

In [6]:
# Fetch Full HISTORY
def fetch_full_historical(total_days: int) -> pd.DataFrame:
    to_ts = int(time.time())
    days_left = total_days
    all_days = []

    while days_left > 0:
        batch = min(HISTORICAL_CHUNK, days_left)
        chunk = fetch_chunk(batch, to_ts)
        if not chunk:
            break

        all_days.extend(chunk)
        to_ts = chunk[0]["time"] - 1
        days_left -= batch
        time.sleep(0.2)  # be kind to the API

    df = pd.DataFrame(all_days)
    df["date"] = pd.to_datetime(df["time"], unit="s").dt.date
    df = df.rename(columns={
        "open": "open_usd",
        "high": "high_usd",
        "low": "low_usd",
        "close": "close_usd",
        "volumeto": "volume_usd"
    })
    return (
        df[["date", "open_usd", "high_usd", "low_usd", "close_usd", "volume_usd"]]
        .drop_duplicates("date")
        .sort_values("date")
        .reset_index(drop=True)
    )



## Storing Data in PostgreSQL

In [9]:
# MAIN
def main():
    try:
        # Number of year of data you want to fetch
        num_years = 12
        total_days = int(num_years * 365.25)
    except ValueError:
        print("Invalid input. Please enter a numeric value.")
        return

    print(f"\nFetching ~{total_days} days of BTC/USD data (~{num_years:.2f} years)...")
    df = fetch_full_historical(total_days)
    print(f"Fetched {len(df)} records. Writing to database…")

    # write to Postgres (will replace any existing table with the same name)
    df.to_sql(TABLE_NAME, engine, if_exists="replace", index=False)
    print(f"✅ Data saved to table '{TABLE_NAME}' in your database.")

    # Optional: preview first few rows
    print(df.head())


if __name__ == "__main__":
    main()


Fetching ~4383 days of BTC/USD data (~12.00 years)...
Fetched 4383 records. Writing to database…
✅ Data saved to table 'bitcion_daily_data' in your database.
         date  open_usd  high_usd  low_usd  close_usd   volume_usd
0  2013-05-12    115.64    117.47   112.40     114.82   2357929.41
1  2013-05-13    114.82    118.88   114.50     117.98   3058207.49
2  2013-05-14    117.98    119.80   109.42     111.40  10075279.73
3  2013-05-15    111.40    116.44   103.02     114.22  12997994.80
4  2013-05-16    114.22    118.97   112.10     118.21   5202992.37


# Live Data Fetch

This notebook demonstrates how to wrap the CryptoCompare “histominute” endpoint to:

1. **Fetch** full historical minute-level Bitcoin data (past 2 days).
2. **Fetch** real-time Bitcoin data in one-day windows.
3. **Append** new observations into an existing CSV.

We’ll build helper functions, show their usage, and finally run a one-off historical download.  


In [3]:
import os
import time
from datetime import datetime

import pandas as pd
import requests
from dotenv import load_dotenv
from sqlalchemy import create_engine, inspect, text

# LOAD ENV VARS
load_dotenv()  # pip install python-dotenv
API_KEY      = os.getenv("API_KEY")
DATABASE_URL = os.getenv("DATABASE_URL")

if not API_KEY:
    raise EnvironmentError("API_KEY missing")
if not DATABASE_URL:
    raise EnvironmentError("DATABASE_URL missing")


# Fix for SQLAlchemy ≥1.4
if DATABASE_URL.startswith("postgres://"):
    DATABASE_URL = DATABASE_URL.replace("postgres://", "postgresql://", 1)

engine = create_engine(DATABASE_URL, echo=False, future=True)

# CONFIG 
TABLE_NAME        = "btc_minute_data123"
HISTORICAL_DAYS   = 7
HISTORICAL_MIN    = HISTORICAL_DAYS * 24 * 60  # total minutes to fetch initially
HISTORICAL_CHUNK  = 2000                      # max per-request limit
REALTIME_LIMIT    = 24 * 60                   # last 24h, single request
MAX_ROWS          = 2000

BASE_URL = "https://min-api.cryptocompare.com/data/v2/histominute"
HEADERS  = {"authorization": f"Apikey {API_KEY}"}


##  Code for fetch_chunk

In [4]:
def fetch_chunk(limit: int, to_ts: int = None) -> list:
    params = {"fsym": "BTC", "tsym": "USD", "limit": limit}
    if to_ts:
        params["toTs"] = to_ts

    r = requests.get(BASE_URL, params=params, headers=HEADERS)
    r.raise_for_status()
    data = r.json()
    if data.get("Response") != "Success":
        raise RuntimeError(f"API Error: {data.get('Message', 'Unknown')}")
    return data["Data"]["Data"]


## Historical Data Fetcher

In [5]:
def fetch_full_historical(total_minutes: int) -> pd.DataFrame:
    to_ts = int(time.time())
    left = total_minutes
    bars = []

    while left > 0:
        batch_size = min(HISTORICAL_CHUNK, left)
        chunk = fetch_chunk(batch_size, to_ts)
        if not chunk:
            break
        bars.extend(chunk)
        to_ts = chunk[0]["time"] - 1
        left -= batch_size
        time.sleep(0.2)

    df = pd.DataFrame(bars)
    df["timestamp"]  = pd.to_datetime(df["time"], unit="s")
    df["price_usd"]  = df["close"]
    df["volume_usd"] = df["volumeto"]
    df["volume_btc"] = df["volumefrom"]

    return (
        df[["timestamp", "price_usd", "volume_usd", "volume_btc"]]
        .drop_duplicates(subset="timestamp")
        .sort_values("timestamp")
        .reset_index(drop=True)
    )

## Real-Time Data Fetcher

In [6]:
def fetch_realtime_df() -> pd.DataFrame:
    bars = fetch_chunk(REALTIME_LIMIT)
    df = pd.DataFrame(bars)
    df["timestamp"]  = pd.to_datetime(df["time"], unit="s")
    df["price_usd"]  = df["close"]
    df["volume_usd"] = df["volumeto"]
    df["volume_btc"] = df["volumefrom"]
    return df[["timestamp", "price_usd", "volume_usd", "volume_btc"]]

## DATABASE UTILITIES

In [7]:
def ensure_table_exists():
    insp = inspect(engine)
    if not insp.has_table(TABLE_NAME):
        print(f"Table '{TABLE_NAME}' not found—inserting initial {HISTORICAL_DAYS}-day history…")
        df0 = fetch_full_historical(HISTORICAL_MIN)
        df0.to_sql(TABLE_NAME, engine, if_exists="replace", index=False)
        print(f"  → Inserted {len(df0)} rows into '{TABLE_NAME}'.")
    else:
        print(f"Table '{TABLE_NAME}' already exists, skipping initial load.")

## REMOVE OLD DATA

In [8]:
def prune_old_rows(conn):
    total = conn.execute(text(f"SELECT COUNT(*) FROM {TABLE_NAME}")).scalar_one()
    if total > MAX_ROWS:
        to_delete = total - MAX_ROWS
        conn.execute(text(f"""
            DELETE FROM {TABLE_NAME}
            WHERE ctid IN (
                SELECT ctid
                FROM {TABLE_NAME}
                ORDER BY timestamp ASC
                LIMIT :n
            )
        """), {"n": to_delete})
        conn.commit()
        print(f"Pruned {to_delete} old rows; now {MAX_ROWS} rows remain.")

Fetching initial 7-day dataset…
Saved 10080 rows to btc_data.csv
[2025-05-12 04:28:09] Fetching last 1440 minutes…
Appended new data. Total rows: 10081
[2025-05-12 04:29:11] Fetching last 1440 minutes…
Appended new data. Total rows: 10082
[2025-05-12 04:30:13] Fetching last 1440 minutes…
Appended new data. Total rows: 10083
[2025-05-12 04:31:14] Fetching last 1440 minutes…
Appended new data. Total rows: 10084
[2025-05-12 04:32:16] Fetching last 1440 minutes…
Appended new data. Total rows: 10085
[2025-05-12 04:33:19] Fetching last 1440 minutes…
Appended new data. Total rows: 10086
[2025-05-12 04:34:21] Fetching last 1440 minutes…
Appended new data. Total rows: 10087
[2025-05-12 04:35:22] Fetching last 1440 minutes…
Appended new data. Total rows: 10088
[2025-05-12 04:36:24] Fetching last 1440 minutes…
Appended new data. Total rows: 10089
[2025-05-12 04:37:25] Fetching last 1440 minutes…
Appended new data. Total rows: 10090
[2025-05-12 04:38:27] Fetching last 1440 minutes…
Appended new da

Appended new data. Total rows: 10203
[2025-05-12 06:31:54] Fetching last 1440 minutes…
Appended new data. Total rows: 10204
[2025-05-12 06:32:55] Fetching last 1440 minutes…
Appended new data. Total rows: 10205
[2025-05-12 06:33:58] Fetching last 1440 minutes…
Appended new data. Total rows: 10206
[2025-05-12 06:35:00] Fetching last 1440 minutes…
Appended new data. Total rows: 10208
[2025-05-12 06:36:02] Fetching last 1440 minutes…
Appended new data. Total rows: 10209
[2025-05-12 06:37:03] Fetching last 1440 minutes…
Appended new data. Total rows: 10210
[2025-05-12 06:38:06] Fetching last 1440 minutes…
Appended new data. Total rows: 10211
[2025-05-12 06:39:08] Fetching last 1440 minutes…
Appended new data. Total rows: 10212
[2025-05-12 06:40:10] Fetching last 1440 minutes…
Appended new data. Total rows: 10213
[2025-05-12 06:41:12] Fetching last 1440 minutes…
Appended new data. Total rows: 10214
[2025-05-12 06:42:13] Fetching last 1440 minutes…
Appended new data. Total rows: 10215
[2025-

ConnectionError: HTTPSConnectionPool(host='min-api.cryptocompare.com', port=443): Max retries exceeded with url: /data/v2/histominute?fsym=BTC&tsym=USD&limit=1440 (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x77e320751280>: Failed to resolve 'min-api.cryptocompare.com' ([Errno -3] Temporary failure in name resolution)"))

## MAIN

In [ ]:
def main():
    ensure_table_exists()

    print("⏱️  Entering realtime update loop (every 60s)…")
    while True:
        now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        print(f"[{now}] Fetching last 24h of bars…", end=" ")

        new_df = fetch_realtime_df()

        with engine.begin() as conn:
            # get latest timestamp in DB
            result = conn.execute(text(f"SELECT MAX(timestamp) FROM {TABLE_NAME}"))
            max_ts = result.scalar_one()
            if max_ts is not None:
                new_df = new_df[new_df["timestamp"] > max_ts]

            if not new_df.empty:
                new_df.to_sql(TABLE_NAME, conn, if_exists="append", index=False)
                # dedupe any overlapping timestamps
                conn.execute(text(f"""
                    DELETE FROM {TABLE_NAME} a
                    USING {TABLE_NAME} b
                    WHERE a.timestamp = b.timestamp
                      AND a.ctid < b.ctid
                """))
                prune_old_rows(conn)
                print(f"Appended {len(new_df)} new rows.")
            else:
                print("No new bars to append.")

        time.sleep(60)


if __name__ == "__main__":
    main()
